In [1]:
import ms4_franklab_pyplines as pyp
import pyff_utils as pyff
import ms4_franklab_proc2py as p2p
import sys, os, subprocess
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
# anim details. for now, assume that sort should include all eps/day
mda_util_path = '/home/anna/Src/MLpackages/franklab_ms_resources/mda_util.py'
ntlinks_path = '/home/anna/Src/MLpackages/franklab_ms_resources/setup_NTlinks.node.js'
base_dir = '/media/anna/whirlwindtemp1/'
anim = 'gus'
days = [1]
dates = [20171022, 20171023, 20171024]

# if datelist not provided, fetch dates from taskinfo struct
# TODO make py_evaluate filter more flexible so that it can do this

#if not dates or len(days)!=len(dates):
#    dates = pyff.py_evaluatefilter(base_dir, anim, 'taskinfo', days,['date','*'])

#if tetlist not provided, fetch ca1 tets from tetinfo struct
tetfilt = ['area','ca1']
tetlist = []
if not tetlist and tetfilt:
    tetlist = pyff.py_evaluatefilter(base_dir, anim, 'tetinfo', days,tetfilt)

In [4]:
tetlist

defaultdict(list, {1: [1, 22, 23, 29]})

In [10]:
for index, day in enumerate(days):

    day_dir = base_dir+anim+'/preprocessing/'+str(dates[index])+'/'
    # check if mnt directory exists for this day. if not, call mda_util to generate the symlinks to the mda files
    mnt_path = day_dir+str(dates[index])+'_'+anim+'.mnt/'
    print('processing day '+str(day))
    if not os.path.exists(mnt_path):
        print('no mnt directory found; calling mda_util')
        os.chdir(base_dir+anim+'/preprocessing')
        subprocess.call(['python',mda_util_path])  
    
    # check if .mountain dir exists for this day
    # if not, run the setup javascript to generate prvs that link mda files (via their symlinks) across epochs
    mountain_path = day_dir+str(dates[index])+'_'+anim+'.mountain'
    if not os.path.exists(mountain_path):
        print('no mountain dir found; calling setup_NTlinks')
        os.chdir(day_dir)
        subprocess.call([ntlinks_path, mnt_path])
    
    #now for the actually processing: iterate through ntrodes
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        
        # concatenate all eps, since ms4 no longer takes a list of mdas; save as raw.mda
        prv_list=nt_dir+'/raw.mda.prv'
        pyp.concat_eps(dataset_dir=nt_dir,mda_list=prv_list)
        
        # preprocessing: filter, mask out artifacts, whiten
        # TODO make optional whether you save the interediates (filt, pre)
        pyp.filt_mask_whiten(dataset_dir=nt_dir,output_dir=nt_dir, freq_min=300,freq_max=6000, opts={})
        
        #run the actual sort 
        pyp.ms4_sort_on_segs(dataset_dir=nt_dir,output_dir=nt_dir, adjacency_radius=-1,detect_threshold=3, detect_sign=-1, opts={})
        
        

processing day 1
RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ms3.mask_out_artifacts --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/filt.mda.prv --parameters interval_size:2000 threshold:5 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and sub

RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/firings_raw.mda timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre.mda.prv --parameters samplerate:30000 --outputs cluster_metrics_out:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_c42348872c10b3e0238286a657e228df25306aaa.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ms3.cluster_metrics already completed. ]
[ Creating output prv for cluster_metrics_out ... ]
[ Done. ]
RUNNING: ml-run-process ms3.isolation_metrics --inputs firings:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt1/firings_raw.mda ti

[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt22/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process pyms.extract_timeseries already completed. ]
[ Done. ]
RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt22/pre-3.mda --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:3 --outputs firings_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt22/firings-3.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ms4alg.sort already completed. ]
[ Done. ]
RUNNING: ml-run-process pyms.anneal_segments --inputs firings_list:/media/anna/whirlwindtemp1/gus/preprocessing

RUNNING: ml-run-process pyms.extract_timeseries --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/pre.mda.prv --parameters t1:0 t2:47461992 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/pre-1.mda
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process pyms.extract_timeseries already completed. ]
[ Done. ]
RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/pre-1.mda --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:3 --outputs firings_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt23/firings-1.mda
[ Getting processor spec

[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_68b82cb55cea455d7a7951b124df5ded4ad895cb.prv ...
Locating /media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_metrics_out_31ccb6359dfd669568a51054095c31dae7924538.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process ms3.combine_cluster_metrics already completed. ]
[ Done. ]
RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:30000 --outputs timeseries_out:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/filt.mda.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking 

[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Process pyms.anneal_segments already completed. ]
[ Creating output prv for dmatrix_out ... ]
[ Creating output prv for dmatrix_templates_out ... ]
[ Creating output prv for k1_dmatrix_out ... ]
[ Creating output prv for k2_dmatrix_out ... ]
[ Done. ]
RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/firings_raw.mda timeseries:/media/anna/whirlwindtemp1/gus/preprocessing/20171022/20171022_gus.mountain/nt29/pre.mda.prv --parameters samplerate:30000 --outputs cluster_metrics_out:/media/anna/whirlwindtemp2/MSjstmp/mountainlab/tmp_short_term/output_cluster_metrics_out_3214901bfcceb12f76683f54e8b1f7c311c5fcac.prv
[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /media/anna/whirlwindtemp1/gus/preproc

In [ ]:
pdb

In [ ]:
# in a separate loop, go back and add tags to the metrics - due to bug in ms3.combine_cluster_metrics
for index, day in enumerate(days):
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.add_curation_tags(dataset_dir=nt_dir,output_dir=nt_dir,opts={})


In [ ]:
# generate marks 
for index, day in enumerate(days):
    for nt_ind, nt in enumerate(tetlist[day]):
        nt_dir = mountain_path+'/nt'+str(nt)
        pyp.extract_marks(dataset_dir=nt_dir,output_dir=nt_dir,opts={})